In [279]:
import pandas as pd
import numpy as np
import datetime as dt

In [253]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)

# Features overview
## Feature long list catagories:

>### Player details (as of most recent week)
- ICT scores
- Selected (by other FPL players)
- Player value


>### Details of fixture
- Relative strength of teams
    - Position in league
    - Team strength stats in teams csv
- Home or away
- Forecasted minutes played

>### Player's recent performance (LW = last week)
- Points
- Goals
- Bonus points
- Red/yellow cards
- Own goals
- Saves (GK only)
- Goals conceded (GK, Def)
- Clean sheets (GK, Def, MF)
- 

>### Player's recent performance (L4W = last 4 weeks)
- Average (and stdev) of 
    - Points
    - Goals
    - Bonus points
    - Red/yellow cards
    - Own goals
    - Saves (GK only)
    - Goals conceded (GK, Def)
    - Clean sheets (GK, Def, MF)


>### Player's performance across the season (TSS = This season)
- Average (and stdev) of 
    - Points
    - Goals
    - Bonus points
    - Red/yellow cards
    - Own goals
    - Saves (GK only)
    - Goals conceded (GK, Def)
    - Clean sheets (GK, Def, MF)



>### Player's historic performance (LSS = last season)
- Average (and stdev) of 
    - Points
    - Goals
    - Bonus points
    - Red/yellow cards
    - Own goals
    - Saves (GK only)
    - Goals conceded (GK, Def)
    - Clean sheets (GK, Def, MF)




# Data addresses

In [112]:
folder_19_20 = "/Users/calumthompson/Documents/Fantasy football/GitHub_data/data/2019-20/"
folder_18_19 = "/Users/calumthompson/Documents/Fantasy football/GitHub_data/data/2018-19/"

In [116]:
team_link = "teams.csv"

In [113]:
fixtures_link = 'https://fixturedownload.com/download/epl-2019-GMTStandardTime.csv'

# Get data 
## 1. The spine of the data is the fixtures list 

In [224]:
Date_format = lambda x: pd.datetime.strptime(x, "%d/%m/%Y %H:%M")
fixtures_RAW = pd.read_csv(fixtures_link,parse_dates = ['Date'], date_parser = Date_format)
fixtures_RAW.head()

,Round Number,Date,Location,Home Team,Away Team,Result
0,1,2019-08-09 20:00:00,Anfield,Liverpool,Norwich,4 - 1
1,1,2019-08-10 12:30:00,London Stadium,West Ham,Man City,0 - 5
2,1,2019-08-10 15:00:00,Vitality Stadium,Bournemouth,Sheffield Utd,1 - 1
3,1,2019-08-10 15:00:00,Turf Moor,Burnley,Southampton,3 - 0
4,1,2019-08-10 15:00:00,Selhurst Park,Crystal Palace,Everton,0 - 0


In [225]:
# gws = fixtures_RAW['Round Number'].unique()
teams = fixtures_RAW['Home Team'].unique()

# fixtures_df = pd.DataFrame(columns = ['Team','GW','Opponent', 'Home?'])
merge = []

for team in teams:
        
    team_record = fixtures_RAW.loc[(fixtures_RAW['Home Team'] == team) | (fixtures_RAW['Away Team'] == team)]

    col_team = np.full(38, team)
    col_GWs = np.arange(1,39)
    col_opponent = np.where(team_record['Home Team'] == team, team_record['Away Team'],team_record['Home Team'])
    col_home = np.where((team_record['Home Team'] == team), 1 , 0)
    col_date = team_record['Date'].dt.date

    team_record = pd.DataFrame({'Team':col_team,'GW':col_GWs,'Fixture_date':col_date,'Opponent':col_opponent,'Home?':col_home})
    
    merge.append(team_record)
    
fixtures_df = pd.concat(merge).reset_index(drop = True)    
    

In [226]:
fixtures_df.head()

,Team,GW,Fixture_date,Opponent,Home?
0,Liverpool,1,2019-08-09,Norwich,1
1,Liverpool,2,2019-08-17,Southampton,0
2,Liverpool,3,2019-08-24,Arsenal,1
3,Liverpool,4,2019-08-31,Burnley,0
4,Liverpool,5,2019-09-14,Newcastle,1


## 2. Map to Vaastav's team ids 

In [374]:
Vaastav_teams = pd.read_csv(folder_19_20+team_link, usecols = ['id','name'])
Team_keys = dict(zip(Vaastav_teams['name'], Vaastav_teams['id']))

In [375]:
fixtures_df['Team_id'] = fixtures_df['Team'].map(Team_keys)
fixtures_df['Opponent_id'] = fixtures_df['Opponent'].map(Team_keys)
fixtures_df.head()

,Team,GW,Fixture_date,Opponent,Home?,Team_id,Opponent_id
0,Liverpool,1,2019-08-09,Norwich,1,10,14
1,Liverpool,2,2019-08-17,Southampton,0,10,16
2,Liverpool,3,2019-08-24,Arsenal,1,10,1
3,Liverpool,4,2019-08-31,Burnley,0,10,5
4,Liverpool,5,2019-09-14,Newcastle,1,10,13


In [376]:
# fixtures_df.loc[fixtures_df['Team'] == 'Wolves']

## 3. Match players to team
As players do not have a team column, join on opponent at given date

In [377]:
merged_gw_cols = pd.read_csv(folder_19_20 + "gws/merged_gw.csv", parse_dates = ['kickoff_time'])
merged_gw_cols['Fixture_date'] = merged_gw_cols['kickoff_time'].dt.date
merged_gw_cols.head()

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,Fixture_date
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,1.1,9.0,2019-08-10 11:30:00+00:00,90,11,0,0,0,0,1,0,23399,5.0,0.0,0.0,0,0,0,0,50,True,0,1,2019-08-10
1,Aaron_Lennon_430,0,0,3,0,0.0,430,3,0,0,0.2,2.0,2019-08-10 14:00:00+00:00,6,16,0,0,0,0,1,0,8105,0.0,3.0,0.0,1,0,0,0,50,True,0,1,2019-08-10
2,Aaron_Mooy_516,0,0,0,0,0.0,516,7,0,0,0.0,0.0,2019-08-10 14:00:00+00:00,0,18,0,0,0,0,1,0,16261,3.0,0.0,0.0,0,0,0,0,50,False,0,1,2019-08-10
3,Aaron_Ramsdale_494,0,0,11,0,0.0,494,2,1,0,1.0,9.8,2019-08-10 14:00:00+00:00,90,15,0,0,0,0,1,2,3091,1.0,1.0,0.0,2,0,0,0,45,True,0,1,2019-08-10
4,Aaron_Wan-Bissaka_122,0,2,34,1,16.1,122,9,0,0,4.9,30.4,2019-08-11 15:30:00+00:00,90,6,0,0,0,0,1,0,1879259,0.0,4.0,2.0,8,0,0,0,55,True,0,1,2019-08-11


In [378]:
players_df = merged_gw_cols[['name','opponent_team', 'minutes', 'Fixture_date']]
players_df = players_df.rename(columns  = {'opponent_team' : 'Opponent_id'})
players_df.head()

,name,Opponent_id,minutes,Fixture_date
0,Aaron_Cresswell_376,11,90,2019-08-10
1,Aaron_Lennon_430,16,6,2019-08-10
2,Aaron_Mooy_516,18,0,2019-08-10
3,Aaron_Ramsdale_494,15,90,2019-08-10
4,Aaron_Wan-Bissaka_122,6,90,2019-08-11


In [390]:
dataset_to_date = pd.merge(fixtures_df,players_df, on = ['Opponent_id','Fixture_date'], how = 'left')
dataset_to_date = dataset_to_date[['name','Fixture_date', 'GW','Team','Team_id','Opponent','Opponent_id','Home?', 'minutes']]
dataset_to_date.head()

,name,Fixture_date,GW,Team,Team_id,Opponent,Opponent_id,Home?,minutes
0,Adam_Lallana_195,2019-08-09,1,Liverpool,10,Norwich,14,1,0.0
1,Adrián_San Miguel del Castillo_526,2019-08-09,1,Liverpool,10,Norwich,14,1,51.0
2,Alex_Oxlade-Chamberlain_193,2019-08-09,1,Liverpool,10,Norwich,14,1,0.0
3,Alisson_Ramses Becker_189,2019-08-09,1,Liverpool,10,Norwich,14,1,38.0
4,Andrew_Robertson_181,2019-08-09,1,Liverpool,10,Norwich,14,1,90.0


In [380]:
# dataset_to_date.groupby(['Team','GW']).count()

In [381]:
# dataset_to_date.loc[dataset_to_date['name'] == 'Hélder_Costa_418']

In [382]:
# wolves_df = dataset_to_date.loc[dataset_to_date['Team'] == 'Wolves']

## 4. Downfill players for weeks that have yet to be completed

In [391]:
downfill_merge = []

for team in dataset_to_date['Team'].unique():
    team_df = dataset_to_date.loc[dataset_to_date['Team'] == team]

    team_players = team_df.loc[team_df['Fixture_date'] < dt.date.today()]
    team_players = team_players.loc[team_players['Fixture_date'] == team_players['Fixture_date'].max()]

    games_to_play = team_df.loc[team_df['Fixture_date'] >= dt.date.today()]['Fixture_date'].to_list()

    merge = []

    for i in games_to_play:

        names_col = team_players['name'].unique()
        downfill_df = pd.DataFrame({'Team': team, 'Fixture_date': i, 'name': names_col}).dropna()
        merge.append(downfill_df)

    team_downfill = pd.concat(merge).reset_index(drop = True)    

    downfill_merge.append(team_downfill)
    
downfill_df = pd.concat(downfill_merge)
downfill_df.head()

,Team,Fixture_date,name
0,Liverpool,2020-06-21,Sadio_Mané_192
1,Liverpool,2020-06-21,Rafael_Camacho_201
2,Liverpool,2020-06-21,Caoimhin_Kelleher_531
3,Liverpool,2020-06-21,Adrián_San Miguel del Castillo_526
4,Liverpool,2020-06-21,Trent_Alexander-Arnold_182


In [401]:
dataset_df= pd.merge(dataset_to_date,downfill, how = 'left',on = ['Team','Fixture_date'])
dataset_df['name'] = np.where(dataset_df['name_x'].isna(), dataset_df['name_y'],dataset_df['name_x']  )
dataset_df = dataset_df.drop(columns = ['name_x','name_y'])
dataset_df['forecast'] = np.where(dataset_df['Fixture_date'] < dt.date.today(), 0 , 1)
dataset_df = dataset_df[['name','forecast','Fixture_date', 'GW','Team','Team_id','Opponent','Opponent_id','Home?', 'minutes']]
dataset_df.head()

,name,forecast,Fixture_date,GW,Team,Team_id,Opponent,Opponent_id,Home?,minutes
0,Adam_Lallana_195,0,2019-08-09,1,Liverpool,10,Norwich,14,1,0.0
1,Adrián_San Miguel del Castillo_526,0,2019-08-09,1,Liverpool,10,Norwich,14,1,51.0
2,Alex_Oxlade-Chamberlain_193,0,2019-08-09,1,Liverpool,10,Norwich,14,1,0.0
3,Alisson_Ramses Becker_189,0,2019-08-09,1,Liverpool,10,Norwich,14,1,38.0
4,Andrew_Robertson_181,0,2019-08-09,1,Liverpool,10,Norwich,14,1,90.0


In [402]:
dataset_df.loc[dataset_df['name'] == 'Hélder_Costa_418']

,name,forecast,Fixture_date,GW,Team,Team_id,Opponent,Opponent_id,Home?,minutes
21242,Hélder_Costa_418,0,2019-08-11,1,Wolves,20,Leicester,9,0,0.0
21265,Hélder_Costa_418,0,2019-08-19,2,Wolves,20,Man Utd,12,1,0.0
21288,Hélder_Costa_418,0,2019-08-25,3,Wolves,20,Burnley,5,1,0.0
21311,Hélder_Costa_418,0,2019-09-01,4,Wolves,20,Everton,8,0,0.0
21334,Hélder_Costa_418,0,2019-09-14,5,Wolves,20,Chelsea,6,1,0.0
21357,Hélder_Costa_418,0,2019-09-22,6,Wolves,20,Crystal Palace,7,0,0.0
21380,Hélder_Costa_418,0,2019-09-28,7,Wolves,20,Watford,18,1,0.0
21403,Hélder_Costa_418,0,2019-10-06,8,Wolves,20,Man City,11,0,0.0
21426,Hélder_Costa_418,0,2019-10-19,9,Wolves,20,Southampton,16,1,0.0
21450,Hélder_Costa_418,0,2019-10-27,10,Wolves,20,Newcastle,13,0,0.0


# GW headers


In [151]:
merged_gw_cols = pd.read_csv(folder_19_20 + "gws/merged_gw.csv")
merged_gw_cols.loc[ merged_gw_cols['name'].str.contains("Drinkwater")]

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
110,Daniel_Drinkwater_120,0,0,0,0,0.0,120,3,0,0,0.0,0.0,2019-08-10T14:00:00Z,0,16,0,0,0,0,1,0,136850,0.0,3.0,0.0,0,0,0,0,45,True,0,1
638,Daniel_Drinkwater_120,0,0,0,0,0.0,120,11,0,0,0.0,0.0,2019-08-17T11:30:00Z,0,1,0,0,0,0,2,0,143926,1.0,2.0,0.0,0,-9812,11951,21763,45,False,0,2
1167,Daniel_Drinkwater_120,0,0,0,0,0.0,120,30,0,0,0.0,0.0,2019-08-25T15:30:00Z,0,20,0,0,0,0,3,0,112048,1.0,1.0,0.0,0,-31036,533,31569,45,False,0,3
1698,Daniel_Drinkwater_120,0,0,0,0,0.0,120,32,0,0,0.0,0.0,2019-08-31T16:30:00Z,0,10,0,0,0,0,4,0,95991,3.0,0.0,0.0,0,-17201,1069,18270,45,True,0,4
2232,Daniel_Drinkwater_120,0,0,0,0,0.0,120,43,0,0,0.0,0.0,2019-09-14T14:00:00Z,0,4,0,0,0,0,5,0,83321,1.0,1.0,0.0,0,-13721,1745,15466,45,False,0,5
2772,Daniel_Drinkwater_120,0,0,0,0,0.0,120,52,0,0,0.0,0.0,2019-09-21T14:00:00Z,0,14,0,0,0,0,6,0,73625,0.0,2.0,0.0,0,-9948,119,10067,45,True,0,6
3315,Daniel_Drinkwater_120,0,0,0,0,0.0,120,61,0,0,0.0,0.0,2019-09-28T14:00:00Z,0,2,0,0,0,0,7,0,65596,2.0,2.0,0.0,0,-8137,146,8283,45,False,0,7
3863,Daniel_Drinkwater_120,0,0,0,0,0.0,120,73,0,0,0.0,0.0,2019-10-05T14:00:00Z,0,8,0,0,0,0,8,0,60476,0.0,1.0,0.0,0,-5234,100,5334,44,True,0,8
4416,Daniel_Drinkwater_120,0,0,0,0,0.0,120,86,0,0,0.0,0.0,2019-10-19T14:00:00Z,0,9,0,0,0,0,9,0,56439,1.0,2.0,0.0,0,-4167,116,4283,44,False,0,9
4972,Daniel_Drinkwater_120,0,0,0,0,0.0,120,93,0,0,0.0,0.0,2019-10-26T16:30:00Z,0,6,0,0,0,0,10,0,54441,4.0,2.0,0.0,0,-2155,58,2213,44,True,0,10


In [41]:
player_gw_cols = pd.read_csv(folder_19_20 + "players/Mohamed_Salah_191/gw.csv")
player_gw_cols.head(50)

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards
0,1,2,40,0,39.7,191,1,1,1,13.4,54.8,2019-08-09T19:00:00Z,90,14,0,0,0,0,1,0,2375465,1,4,39.0,12,0,0,0,125,True,0
1,0,0,0,1,11.8,191,19,0,0,4.3,2.4,2019-08-17T14:00:00Z,78,16,0,0,0,0,2,0,2778082,2,1,29.0,3,114282,156910,42628,125,False,0
2,0,3,43,0,16.6,191,24,1,2,15.5,71.2,2019-08-24T16:30:00Z,90,1,0,0,0,0,3,0,2444065,1,3,67.0,15,-351541,47097,398638,125,True,0
3,1,0,15,1,38.4,191,32,0,0,12.3,27.0,2019-08-31T16:30:00Z,90,5,0,0,0,0,4,0,2810462,3,0,58.0,6,243530,291554,48024,125,False,0
4,0,2,34,0,41.8,191,44,1,1,15.9,44.6,2019-09-14T11:30:00Z,90,13,0,0,0,0,5,0,2874525,1,3,73.0,9,31898,124574,92676,125,True,0
5,1,0,11,0,15.3,191,53,1,0,6.6,14.8,2019-09-22T15:30:00Z,90,6,0,0,0,0,6,0,2804643,2,1,36.0,5,-80384,49872,130256,125,False,0
6,0,0,2,1,13.3,191,68,0,0,8.9,7.6,2019-09-28T11:30:00Z,90,15,0,0,0,0,7,0,2889458,1,0,68.0,3,48879,123616,74737,125,False,0
7,0,0,5,0,37.7,191,74,1,0,15.0,18.2,2019-10-05T14:00:00Z,90,9,0,0,0,0,8,0,2797360,1,2,94.0,2,-92287,34441,126728,125,True,0
8,0,0,0,0,0.0,191,87,0,0,0.0,0.0,2019-10-20T15:30:00Z,0,12,0,0,0,0,9,0,2186184,1,1,0.0,0,-601424,13726,615150,124,False,0
9,0,0,25,0,15.3,191,94,1,1,15.3,45.0,2019-10-27T16:30:00Z,84,17,0,0,0,0,10,0,2096834,1,2,93.0,7,-114968,23153,138121,124,True,0
